In [2]:
import gmplot
import pandas as pd

In [106]:
if __name__ == "__main__": 
     
    file = open('C:\\Users\\S\\Downloads\\wimt_full_seperated_route_map_merged.txt',"r")
    
    gmap1 = gmplot.GoogleMapPlotter(21.1458,79.0882, 5 )  
    route_lat,route_lon = [],[]
    count,name = 0,""
    next(file)
    train_no = 1
    for line in file:  
        x = int(line.split('\t')[0])
        prev_name = name
        name = (line.split('\t')[1])
        
        if count == 0:
            count+=1
            train_no = int(x)
            try:
                route_lat.append(float(line.split('\t')[11]))
                route_lon.append(float(line.split('\t')[12][:-1]))
            except:
                continue #only for spaces
                
        else:
            if x == train_no:
                try:
                    route_lat.append(float(line.split('\t')[11]))
                    route_lon.append(float(line.split('\t')[12][:-1]))
                except:
                    continue #only for spaces
            else:
                train_no = int(x)
                
                gmap1.plot(route_lat, route_lon, color = 'blue', edge_width = 2)
                
                route_lat,route_lon = [],[]
                
                try:
                    route_lat.append(float(line.split('\t')[11]))
                    route_lon.append(float(line.split('\t')[12][:-1]))
                except:
                    continue #only for spaces
        
        count+=1       
        
    gmap1.plot(route_lat, route_lon, color = 'blue', edge_width = 2) 

In [107]:
import shapely
import math
from shapely.geometry import Point
from shapely.geometry import LineString
from math import radians, cos, sin, asin, sqrt

def dist_bw2points(lat1,lon1,lat2,lon2):
    # to calculate Distance Between Two Points on Earth

    # converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles

    return (c*r)        
        
def error_check(route_lat,route_lon,points,station_names,csv_file) :
    csv_file.write(station_names[0]+ "," + "0" + '\n')
    for i in range(1,len(route_lat)):
        if i == len(route_lat) -1:
            csv_file.write(station_names[len(route_lat)-1]+ "," + "0" + '\n')
        else:    
            point = Point(route_lat[i], route_lon[i])
            dist = LineString([points[0],points[1]]).project(point)
            l = list(LineString([points[0],points[1]]).interpolate(dist).coords)
            err_dist = dist_bw2points(route_lat[i],route_lon[i],l[0][0],l[0][1])
            csv_file.write(station_names[i]+ "," + str(err_dist) + '\n')
    

def improve(route_lat,route_lon,err_dist,station_names,gmap1,csv_file):
    junc_lat,junc_lon,junc_names,i = [route_lat[0]],[route_lon[0]],[station_names[0]],0
    i = 1
    while i < len(err_dist):        
        while i < len(err_dist) and err_dist[i] >= 0 and err_dist[i] <= 30:
            i+=1  
        if i < len(err_dist) and err_dist[i] > 30:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])     
            junc_names.append(station_names[i])
        maxi1,index1 = 0,0    
        while i < len(err_dist) and err_dist[i] > 30 and err_dist[i] <= 60:
            if maxi1 < err_dist[i]:
                maxi1 = err_dist[i]
                index1 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 60:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index1>0:
            junc_lat.append(route_lat[index1])
            junc_lon.append(route_lon[index1])
            junc_names.append(station_names[i])
            
        maxi2,index2 = 0,0            
        while i < len(err_dist) and err_dist[i] > 60 and err_dist[i] <= 90:
            if maxi2 < err_dist[i]:
                maxi2 = err_dist[i]
                index2 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 90:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index2 > 0:
            junc_lat.append(route_lat[index2])
            junc_lon.append(route_lon[index2]) 
            junc_names.append(station_names[i])
            
    num_of_lines = 0        
    if i == len(err_dist):     
        junc_lat.append(route_lat[len(err_dist)-1])
        junc_lon.append(route_lon[len(err_dist)-1])
        junc_names.append(station_names[len(err_dist)-1])
        
        gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        num_of_lines = len(junc_lat)-1
        j = 1
        points = []
        points.append((route_lat[0] ,route_lon[0]))
        start = 0
        for j in range(1,len(route_lat)):
            if (len(points) == 1) and (station_names[j] in junc_names):
                points.append((route_lat[j],route_lon[j]))
                end = j
                error_check(route_lat[start:end+1], route_lon[start:end+1],points,station_names[start:end+1],csv_file)
                points = []
                points.append((route_lat[j],route_lon[j]))
                start = j
        
        if len(points) == 2:
            end = j-1
            error_check(route_lat[start:end+1], route_lon[start:end+1],points,station_names[start:end+1],csv_file)
                
        gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)
        return num_of_lines
    
if __name__ == "__main__":
    
    csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\demofile2.txt", "a")
    csv_file.write("station_code" + "," + "err_dist" + '\n')
    file = open('C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\error_dist_junc_to_junc_data.txt',"r")
    count,sr_no,name,route_lat,route_lon,station_names,err_dst = 0,0,"",[],[],[],[]
    next(file)
    name, prev_n = "",""
    sum_route_lines = 0
    for line in file:  
        x = int(line.split(',')[0])
        prev_n = name
        name = line.split(',')[1]
        lati,longi,err = float(line.split(',')[3]),float(line.split(',')[4][:-1]),float(line.split(',')[2])
        if count == 0:
            count+=1
            train_no = int(x)
            print(x)
            route_lat.append(lati)
            route_lon.append(longi)
            station_names.append(name)
            err_dst.append(err)
        else:
            if x == train_no:
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
            else:
                train_no = int(x)
                #print("called")
                x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
                sum_route_lines += x
                route_lat, route_lon,err_dst,station_names = [],[],[],[]
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
        count+=1 
    if len(route_lat) > 0:
        x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
        sum_route_lines += x
    file.close()
    gmap1.draw( "C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(threshold_30).html" )
    print(sum_route_lines)

0
1543


In [ ]:
Google_Project\\error_dist_junc_to_junc_data

In [105]:
def improve(route_lat,route_lon,err_dist,gmap1):
    junc_lat,junc_lon,i = [route_lat[0]],[route_lon[0]],0
    
    while i < len(err_dist):        
        while i < len(err_dist) and err_dist[i] >= 0 and err_dist[i] <= 20:
            i+=1  
        if i < len(err_dist) and err_dist[i] > 20:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])     
            
        maxi1,index1 = 0,0    
        while i < len(err_dist) and err_dist[i] > 20 and err_dist[i] <= 40:
            if maxi1 < err_dist[i]:
                maxi1 = err_dist[i]
                index1 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 40:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index1>0:
            junc_lat.append(route_lat[index1])
            junc_lon.append(route_lon[index1])
            
        maxi2,index2 = 0,0    
        while i < len(err_dist) and err_dist[i] > 40 and err_dist[i] <= 60:
            if maxi2 < err_dist[i]:
                maxi2 = err_dist[i]
                index2 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 60:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index2>0:
            junc_lat.append(route_lat[index2])
            junc_lon.append(route_lon[index2])    
            
        maxi3,index3 = 0,0            
        while i < len(err_dist) and err_dist[i] > 60 and err_dist[i] <= 80:
            if maxi3 < err_dist[i]:
                maxi3 = err_dist[i]
                index3 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 80:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index3 > 0:
            junc_lat.append(route_lat[index3])
            junc_lon.append(route_lon[index3])    
        
        maxi4,index4 = 0,0            
        while i < len(err_dist) and err_dist[i] > 80 and err_dist[i] <= 100:
            if maxi4 < err_dist[i]:
                maxi4 = err_dist[i]
                index4 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 100:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index4 > 0:
            junc_lat.append(route_lat[index4])
            junc_lon.append(route_lon[index4])
        
    num_of_lines = 0        
    if i == len(err_dist):     
        junc_lat.append(route_lat[len(err_dist)-1])
        junc_lon.append(route_lon[len(err_dist)-1])
        gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        num_of_lines = len(junc_lat)-1
        gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)
        return num_of_lines
    
if __name__ == "__main__":
    
    
    file = open('C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\error_dist_junc_to_junc_data.txt',"r")
    count,sr_no,name,route_lat,route_lon,err_dst = 0,0,"",[],[],[]
    next(file)
    name, prev_n = "",""
    sum_route_lines = 0
    for line in file:  
        x = int(line.split(',')[0])
        prev_n = name
        name = line.split(',')[1]
        lati,longi,err = float(line.split(',')[3]),float(line.split(',')[4][:-1]),float(line.split(',')[2])
        if count == 0:
            count+=1
            train_no = int(x)
            print(x)
            route_lat.append(lati)
            route_lon.append(longi)
            err_dst.append(err)
        else:
            if x == train_no:
                route_lat.append(float(line.split(',')[3]))
                route_lon.append(float(line.split(',')[4][:-1]))
                err_dst.append(float(line.split(',')[2]))
            else:
                train_no = int(x)
                #print("called")
                x = improve(route_lat,route_lon,err_dst,gmap1)
                sum_route_lines += x
                route_lat, route_lon,err_dst = [],[],[]
                route_lat.append(float(line.split(',')[3]))
                route_lon.append(float(line.split(',')[4][:-1]))
                err_dst.append(float(line.split(',')[2]))
        count+=1 
    if len(route_lat) > 0:
        x = improve(route_lat,route_lon,err_dst,gmap1)
        sum_route_lines += x
    file.close()
    gmap1.draw( "C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(threshold_20).html" )
    print(sum_route_lines)

0
1669


In [97]:
def improve(route_lat,route_lon,err_dist,gmap1):
    junc_lat,junc_lon,i = [route_lat[0]],[route_lon[0]],0
    
    while i < len(err_dist):        
        while i < len(err_dist) and err_dist[i] >= 0 and err_dist[i] <= 10:
            i+=1  
        if i < len(err_dist) and err_dist[i] > 10:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])     
            
        maxi1,index1 = 0,0    
        while i < len(err_dist) and err_dist[i] > 10 and err_dist[i] <= 20:
            if maxi1 < err_dist[i]:
                maxi1 = err_dist[i]
                index1 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 20:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index1>0:
            junc_lat.append(route_lat[index1])
            junc_lon.append(route_lon[index1])
            
        maxi2,index2 = 0,0    
        while i < len(err_dist) and err_dist[i] > 20 and err_dist[i] <= 30:
            if maxi2 < err_dist[i]:
                maxi2 = err_dist[i]
                index2 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 30:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index2>0:
            junc_lat.append(route_lat[index2])
            junc_lon.append(route_lon[index2])    
            
        maxi3,index3 = 0,0            
        while i < len(err_dist) and err_dist[i] > 30 and err_dist[i] <= 40:
            if maxi3 < err_dist[i]:
                maxi3 = err_dist[i]
                index3 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 40:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index3 > 0:
            junc_lat.append(route_lat[index3])
            junc_lon.append(route_lon[index3])    
        
        maxi4,index4 = 0,0            
        while i < len(err_dist) and err_dist[i] > 40 and err_dist[i] <= 50:
            if maxi4 < err_dist[i]:
                maxi4 = err_dist[i]
                index4 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 50:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index4 > 0:
            junc_lat.append(route_lat[index4])
            junc_lon.append(route_lon[index4])
            
        maxi5,index5 = 0,0            
        while i < len(err_dist) and err_dist[i] > 50 and err_dist[i] <= 60:
            if maxi5 < err_dist[i]:
                maxi5 = err_dist[i]
                index5 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 60:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index5 > 0:
            junc_lat.append(route_lat[index5])
            junc_lon.append(route_lon[index5])   
            
            
        maxi6,index6 = 0,0            
        while i < len(err_dist) and err_dist[i] > 60 and err_dist[i] <= 70:
            if maxi6 < err_dist[i]:
                maxi6 = err_dist[i]
                index6 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 70:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index6 > 0:
            junc_lat.append(route_lat[index6])
            junc_lon.append(route_lon[index6])
            
            
        maxi7,index7 = 0,0            
        while i < len(err_dist) and err_dist[i] > 70 and err_dist[i] <= 80:
            if maxi7 < err_dist[i]:
                maxi7 = err_dist[i]
                index7 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 70:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index7 > 0:
            junc_lat.append(route_lat[index7])
            junc_lon.append(route_lon[index7])
            
            
        maxi8,index8 = 0,0            
        while i < len(err_dist) and err_dist[i] > 80 and err_dist[i] <= 90:
            if maxi8 < err_dist[i]:
                maxi8 = err_dist[i]
                index8 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 80:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
        elif i < len(err_dist) and index8 > 0:
            junc_lat.append(route_lat[index8])
            junc_lon.append(route_lon[index8])    
        
    num_of_lines = 0        
    if i == len(err_dist):     
        junc_lat.append(route_lat[len(err_dist)-1])
        junc_lon.append(route_lon[len(err_dist)-1])
        gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        num_of_lines = len(junc_lat)-1
        gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)
        return num_of_lines
    
if __name__ == "__main__":
    
    
    file = open('C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\error_dist_junc_to_junc_data.txt',"r")
    count,sr_no,name,route_lat,route_lon,err_dst = 0,0,"",[],[],[]
    next(file)
    name, prev_n = "",""
    sum_route_lines = 0
    for line in file:  
        x = int(line.split(',')[0])
        prev_n = name
        name = line.split(',')[1]
        lati,longi,err = float(line.split(',')[3]),float(line.split(',')[4][:-1]),float(line.split(',')[2])
        if count == 0:
            count+=1
            train_no = int(x)
            print(x)
            route_lat.append(lati)
            route_lon.append(longi)
            err_dst.append(err)
        else:
            if x == train_no:
                route_lat.append(float(line.split(',')[3]))
                route_lon.append(float(line.split(',')[4][:-1]))
                err_dst.append(float(line.split(',')[2]))
            else:
                train_no = int(x)
                #print("called")
                x = improve(route_lat,route_lon,err_dst,gmap1)
                sum_route_lines += x
                route_lat, route_lon,err_dst = [],[],[]
                route_lat.append(float(line.split(',')[3]))
                route_lon.append(float(line.split(',')[4][:-1]))
                err_dst.append(float(line.split(',')[2]))
        count+=1 
    if len(route_lat) > 0:
        x = improve(route_lat,route_lon,err_dst,gmap1)
        sum_route_lines += x
    file.close()
    gmap1.draw( "C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(threshold_10).html" )
    print(sum_route_lines)

0
2320


In [95]:
import shapely
import math
from shapely.geometry import Point
from shapely.geometry import LineString
from math import radians, cos, sin, asin, sqrt

def dist_bw2points(lat1,lon1,lat2,lon2):
    # to calculate Distance Between Two Points on Earth

    # converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles

    return (c*r)        
        
def error_check(route_lat,route_lon,points,station_names,csv_file) :
    csv_file.write(station_names[0]+ "," + "0" + '\n')
    for i in range(1,len(route_lat)):
        if i == len(route_lat) -1:
            csv_file.write(station_names[len(route_lat)-1]+ "," + "0" + '\n')
        else:    
            point = Point(route_lat[i], route_lon[i])
            dist = LineString([points[0],points[1]]).project(point)
            l = list(LineString([points[0],points[1]]).interpolate(dist).coords)
            err_dist = dist_bw2points(route_lat[i],route_lon[i],l[0][0],l[0][1])
            csv_file.write(station_names[i]+ "," + str(err_dist) + '\n')
    

def improve(route_lat,route_lon,err_dist,station_names,gmap1,csv_file):
    l = len(route_lat) - 1
    num_of_lines = 1
    maxi = max(err_dist)
    junc_lon,junc_lat=[],[]
    if maxi > 20:
        index = err_dist.index(maxi)
        points = []
        junc_lon,junc_lat=[],[]
        junc_lat = [route_lat[0],route_lat[index]]
        junc_lon = [route_lon[0],route_lon[index]]
        points.append((route_lat[0],route_lon[0]))
        points.append((route_lat[index],route_lon[index]))
        error_check(route_lat[:index+1],route_lon[:index+1],points,station_names[:index+1],csv_file)
        gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)

        points = []
        junc_lon,junc_lat=[],[]
        junc_lat = [route_lat[index],route_lat[l]]
        junc_lon = [route_lon[index],route_lon[l]]
        points.append((route_lat[index] ,route_lon[index]))
        points.append((route_lat[l],route_lon[l]))
        error_check(route_lat[index:l+1],route_lon[index:l+1],points,station_names[index:l+1],csv_file)
        gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)

        num_of_lines = 2
    else:
        junc_lon,junc_lat=[],[]
        junc_lat = [route_lat[0],route_lat[l]]
        junc_lon = [route_lon[0],route_lon[l]]
        gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)
            
    return num_of_lines
    
if __name__ == "__main__":
    
    gmap1 = gmplot.GoogleMapPlotter(21.1458,79.0882, 5 )
    csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\demofile3.txt", "a")
    csv_file.write("station_code" + "," + "err_dist" + '\n')
    file = open('C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\error_dist_junc_to_junc_data.txt',"r")
    count,sr_no,name,route_lat,route_lon,station_names,err_dst = 0,0,"",[],[],[],[]
    next(file)
    name, prev_n = "",""
    sum_route_lines = 0
    for line in file:  
        x = int(line.split(',')[0])
        prev_n = name
        name = line.split(',')[1]
        lati,longi,err = float(line.split(',')[3]),float(line.split(',')[4][:-1]),float(line.split(',')[2])
        if count == 0:
            count+=1
            train_no = int(x)
            print(x)
            route_lat.append(lati)
            route_lon.append(longi)
            station_names.append(name)
            err_dst.append(err)
        else:
            if x == train_no:
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
            else:
                train_no = int(x)
                #print("called")
                x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
                sum_route_lines += x
                route_lat, route_lon,err_dst,station_names = [],[],[],[]
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
        count+=1 
    if len(route_lat) > 0:
        x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
        sum_route_lines += x
    file.close()
    gmap1.draw( "C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(midpoint).html" )
    print(sum_route_lines)

0
1568


In [3]:
import gmplot
import pandas as pd
import shapely
import math
from shapely.geometry import Point
from shapely.geometry import LineString
from math import radians, cos, sin, asin, sqrt


class Graph:
    
    def __init__(self):
        self.adjacency_list = dict()
        
    def add_edge(self,src,dest):
        if src not in self.adjacency_list:
            self.adjacency_list[src] = []
        self.adjacency_list[src].append(dest)
        
        if dest not in self.adjacency_list:
            self.adjacency_list[dest] = []
        self.adjacency_list[dest].append(src)
        
    def print_graph(self):
        for x in self.adjacency_list:
            print (x , "->" , self.adjacency_list[x])
            
    def find_degree(self,val):
        if val in self.adjacency_list:
            l = set(self.adjacency_list[val])
            return len(l)
        else:
            return 0

def filter_list(L):
    return [x for x in L if not any(set(x)<=set(y) for y in L if x is not y)]

def dist_bw2points(lat1,lon1,lat2,lon2):
    # to calculate Distance Between Two Points on Earth

    # converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    
    return (c*r)

def error_check(route_lat,route_lon,points,station_names,csv_file) :
    sum_error_dist,avg_err = 0,0
    line_joining_two_junctions = LineString([points[0],points[1]])
    csv_file.write(station_names[0] + "," + '0' + '\n')
    for i in range(len(route_lat)): 
        point = Point(route_lat[i], route_lon[i])
        dist = line_joining_two_junctions.project(point)
        lis = list(line_joining_two_junctions.interpolate(dist).coords)
        err_dist = dist_bw2points(route_lat[i],route_lon[i],lis[0][0],lis[0][1])
        csv_file.write(station_names[i+1] + "," + str(err_dist) + '\n')
        sum_error_dist += err_dist
    csv_file.write(station_names[len(station_names)-1] +  "," + '0' + '\n')    
    if len(route_lat) > 0:
        avg_err = sum_error_dist / len(route_lat)
            
    csv_file.write(station_names[0] + "," + str(avg_err) + "," + station_names[len(station_names)-1]+ ","  + '\n')
   
    csv_file.write(" " + "," + " " + "," + " " + '\n')
    return avg_err
    
if __name__ == "__main__":
    
    g = Graph()
    
    data = pd.read_csv('C:\\Users\\S\\Downloads\\station_info_aggregated.txt', sep='|')
    vertices = data['station_code'].tolist()
    
    data['lat'] = data['lat'].astype(float)
    data['lon'] = data['lon'].astype(float)
    
    latitudes = data['lat'].tolist()
    longitudes = data['lon'].tolist()
    
    file = open('C:\\Users\\S\\Downloads\\wimt_full_seperated_route_map_merged.txt',"r")
    count = 0
    next(file)
    train_no = 1
    name, prev_n = "",""
    for line in file:  
        x = int(line.split('\t')[0])
        prev_n = name
        name = line.split('\t')[1]
        if count == 0:
            count+=1
            train_no = int(x)
        else:
            if x == train_no:
                g.add_edge(name,prev_n)
            else:
                train_no = int(x)
        count+=1
       
    junc_endpoint = {}
    for i in range(len(vertices)):
        x = g.find_degree(vertices[i])
        if x > 2 or x == 1 :
            junc_endpoint[vertices[i]] = i
            
    junc_to_junc_stations_list = []
    file.seek(0,0)
    points_names,stations_btw_two_junc = [],[]
    count,name = 0,""
    train_no = 1
    for line in file:  
        x = int(line.split('\t')[0])
        prev_name = name
        name = line.split('\t')[1]
        if count == 0:
            count+=1
            train_no = int(x)
            if name in junc_endpoint:
                points_names.append(name)
                stations_btw_two_junc.append(name)
        else:
            if x == train_no:
                if name in junc_endpoint:
                    points_names.append(name)
                    stations_btw_two_junc.append(name)
                    if len(points_names) == 2 :
                        l = stations_btw_two_junc.copy()
                        l.reverse()
                        if (stations_btw_two_junc not in junc_to_junc_stations_list) and (l not in junc_to_junc_stations_list) :
                            junc_to_junc_stations_list.append(stations_btw_two_junc)
                            
                        points_names,stations_btw_two_junc = [],[]
                        points_names.append(name)
                        stations_btw_two_junc.append(name)
                else:
                    stations_btw_two_junc.append(name)
                    
            else:
                
                train_no = int(x)
               
                points_names,stations_btw_two_junc = [],[]
                if name in junc_endpoint:
                    points_names.append(name)
                    stations_btw_two_junc.append(name)
                
                    
        count+=1 
        
    filter_list(junc_to_junc_stations_list)
    
    csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\avg_error_btw_junctions(indep_train)updated1.txt", "w")
    csv_file.write("First_junc_name" + "," + "Second_junc_name"  + "," + "err_dist" + '\n')
    
    gmap1 = gmplot.GoogleMapPlotter(21.1458,79.0882, 5 )
    
    for i in range(len(junc_to_junc_stations_list)) :
        points,routes_lat,routes_lon = [],[],[]
        lat,lon = [],[]
        for j in range(len(junc_to_junc_stations_list[i])):
            name = junc_to_junc_stations_list[i][j]
            index = vertices.index(name)
            if j == 0 or j == len(junc_to_junc_stations_list[i])-1:
                points.append((latitudes[index] , longitudes[index]))
                lat.append(latitudes[index])
                lon.append(longitudes[index])
            else:
                routes_lat.append(latitudes[index])
                routes_lon.append(longitudes[index])
                
        error_check(routes_lat, routes_lon,points,junc_to_junc_stations_list[i],csv_file) 
        gmap1.scatter(lat,lon,'red', size = 100, marker = False)
        gmap1.plot(lat,lon,color = 'yellow', edge_width = 2)
    gmap1.draw("C:\\Users\\S\\OneDrive\\Desktop\\removed_subsets.html")
    
    csv_file.close()

In [ ]:
import shapely
import math
from shapely.geometry import Point
from shapely.geometry import LineString
from math import radians, cos, sin, asin, sqrt

def dist_bw2points(lat1,lon1,lat2,lon2):
    # to calculate Distance Between Two Points on Earth

    # converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles

    return (c*r)        
        
def error_check(route_lat,route_lon,points,station_names,csv_file) :
    #csv_file.write(station_names[0]+ "," + "0" + '\n')
    sum_err_dist = 0
    for i in range(1,len(route_lat)-1):
        if i == len(route_lat) -1:
            csv_file.write(station_names[len(route_lat)-1]+ "," + "0" + '\n')
        else:    
            point = Point(route_lat[i], route_lon[i])
            dist = LineString([points[0],points[1]]).project(point)
            l = list(LineString([points[0],points[1]]).interpolate(dist).coords)
            err_dist = dist_bw2points(route_lat[i],route_lon[i],l[0][0],l[0][1])
            sum_err_dist+=err_dist
            csv_file.write(station_names[i]+ "," + str(err_dist) + '\n')
    #csv_file.write(station_names[0]+ "," + str(sum_err_dist) + "," + station_names[len(route_lat)-1] + '\n')

def improve(route_lat,route_lon,err_dist,station_names,gmap1,csv_file):
    junc_lat,junc_lon,junc_names,i = [route_lat[0]],[route_lon[0]],[station_names[0]],0
    i = 1
    while i < len(err_dist):        
        while i < len(err_dist) and err_dist[i] >= 0 and err_dist[i] <= 15:
            i+=1  
        if i < len(err_dist) and err_dist[i] > 15:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])     
            junc_names.append(station_names[i])
        maxi1,index1 = 0,0    
        while i < len(err_dist) and err_dist[i] > 15 and err_dist[i] <= 30:
            if maxi1 < err_dist[i]:
                maxi1 = err_dist[i]
                index1 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 30:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index1>0:
            junc_lat.append(route_lat[index1])
            junc_lon.append(route_lon[index1])
            junc_names.append(station_names[i])
            
        maxi2,index2 = 0,0
        while i < len(err_dist) and err_dist[i] > 30 and err_dist[i] <= 45:
            if maxi2 < err_dist[i]:
                maxi2 = err_dist[i]
                index2 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 45:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index2>0:
            junc_lat.append(route_lat[index2])
            junc_lon.append(route_lon[index2])
            junc_names.append(station_names[i])
        
        maxi3,index3 = 0,0
        while i < len(err_dist) and err_dist[i] > 45 and err_dist[i] <= 60:
            if maxi3 < err_dist[i]:
                maxi3 = err_dist[i]
                index3 = i
            i+=1
         
        if i < len(err_dist) and err_dist[i] > 60:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index3>0:
            junc_lat.append(route_lat[index3])
            junc_lon.append(route_lon[index3])
            junc_names.append(station_names[i])
            
        maxi4,index4 = 0,0            
        while i < len(err_dist) and err_dist[i] > 60 and err_dist[i] <= 75:
            if maxi4 < err_dist[i]:
                maxi4 = err_dist[i]
                index4 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 75:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index4 > 0:
            junc_lat.append(route_lat[index4])
            junc_lon.append(route_lon[index4]) 
            junc_names.append(station_names[i])
            
            
        maxi5,index5 = 0,0            
        while i < len(err_dist) and err_dist[i] > 75 and err_dist[i] <= 90:
            if maxi5 < err_dist[i]:
                maxi5 = err_dist[i]
                index5 = i
            i+=1
        if i < len(err_dist) and err_dist[i] > 90:
            junc_lat.append(route_lat[i])
            junc_lon.append(route_lon[i])
            junc_names.append(station_names[i])
        elif i < len(err_dist) and index5 > 0:
            junc_lat.append(route_lat[index5])
            junc_lon.append(route_lon[index5]) 
            junc_names.append(station_names[i])    
            
    num_of_lines = 0        
    if i == len(err_dist):     
        junc_lat.append(route_lat[len(err_dist)-1])
        junc_lon.append(route_lon[len(err_dist)-1])
        junc_names.append(station_names[len(err_dist)-1])
        
        gmap1.scatter(junc_lat,junc_lon,'red', size = 500, marker = False)
        num_of_lines = len(junc_lat)-1
        j = 1
        points = []
        points.append((route_lat[0] ,route_lon[0]))
        start = 0
        for j in range(1,len(route_lat)):
            if (len(points) == 1) and (station_names[j] in junc_names):
                points.append((route_lat[j],route_lon[j]))
                end = j
                error_check(route_lat[start:end+1], route_lon[start:end+1],points,station_names[start:end+1],csv_file)
                points = []
                points.append((route_lat[j],route_lon[j]))
                start = j
        
        if len(points) == 2:
            end = j-1
            error_check(route_lat[start:end+1], route_lon[start:end+1],points,station_names[start:end+1],csv_file)
                
        gmap1.plot(junc_lat,junc_lon,color='yellow',edge_width = 2)
        return num_of_lines
    
if __name__ == "__main__":
    
    csv_file = open("C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\threshold_15_junc_to_jun_data.txt", "w")
    csv_file.write("station_code" + "," + "err_dist" + '\n')
    file = open('C:\\Users\\S\\OneDrive\\Desktop\\Google_Project\\error_dist_junc_to_junc_data.txt',"r")
    count,sr_no,name,route_lat,route_lon,station_names,err_dst = 0,0,"",[],[],[],[]
    next(file)
    name, prev_n = "",""
    sum_route_lines = 0
    for line in file:  
        x = int(line.split(',')[0])
        prev_n = name
        name = line.split(',')[1]
        lati,longi,err = float(line.split(',')[3]),float(line.split(',')[4][:-1]),float(line.split(',')[2])
        if count == 0:
            count+=1
            train_no = int(x)
            print(x)
            route_lat.append(lati)
            route_lon.append(longi)
            station_names.append(name)
            err_dst.append(err)
        else:
            if x == train_no:
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
            else:
                train_no = int(x)
                #print("called")
                x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
                sum_route_lines += x
                route_lat, route_lon,err_dst,station_names = [],[],[],[]
                route_lat.append(lati)
                route_lon.append(longi)
                station_names.append(name)
                err_dst.append(err)
        count+=1 
    if len(route_lat) > 0:
        x = improve(route_lat,route_lon,err_dst,station_names,gmap1,csv_file)
        sum_route_lines += x
    file.close()
    csv_file.close()
    gmap1.draw( "C:\\Users\\S\\OneDrive\\Desktop\\improved_only_new_routes(threshold_15).html" )
    print(sum_route_lines)

0
